In [183]:
import pandas as pd
import re
import requests as r
from bs4 import BeautifulSoup
import re
import nltk
from string import punctuation
from unicodedata import normalize

In [184]:
text = r.get('https://www.mg.superesportes.com.br/futebol/atletico-mg/').text

In [185]:
html = BeautifulSoup(text)
notices = []

In [186]:
list_notices = html.find_all('div', class_='l-generic')

In [187]:
for notice in list_notices:
    if(notice.find_all('script') == []):
        info_notice = []
        info_notice.append(notice.find_all('p')[0].text)
        info_notice.append(notice.find_all('p')[1].text)
        info_notice.append(notice.find_all('b')[0].text)
        info_notice.append(notice.find_all('time')[0].text)
        info_notice.append(notice.find_all('a')[0]['href'])
    
        notices.append(info_notice)

In [188]:
len(notices)

18

In [189]:
df_notices = pd.DataFrame(
    notices,
    columns=['titulo', 'descricao', 'categoria', 'data-hora-publicacao', 'link']
)

In [190]:
df_notices.head()

,titulo,descricao,categoria,data-hora-publicacao,link
0,Lista liderada por Franck Ribéry conta com jog...,"De acordo com ranking do site Transfermarkt, M...",MAIS VALIOSOS,31/08/2019 - 19:04,https://www.mg.superesportes.com.br/app/notici...
1,"Com todos os titulares em campo, Atlético faz ...",Galo deve ter equipe completa no duelo contra ...,ATLÉTICO,31/08/2019 - 12:48,https://www.mg.superesportes.com.br/app/notici...
2,"Time mais disciplinado do Brasileiro, Atlético...",Entre atletas com dois amarelos estão os titul...,ATLÉTICO,31/08/2019 - 07:00,https://www.mg.superesportes.com.br/app/notici...
3,Elias explica como o Atlético deve jogar para ...,Galo vai enfrentar o time paulista neste domin...,ATLÉTICO,31/08/2019 - 06:00,https://www.mg.superesportes.com.br/app/notici...
4,"Por temor de punição, clubes da Série A lançam...","De acordo com o STJD, cânticos homofóbicos são...",FUTEBOL NACIONAL,30/08/2019 - 18:35,https://www.mg.superesportes.com.br/app/notici...


In [191]:
df_notices['doc'] = df_notices.apply(lambda row : row['titulo'] + row['descricao'], axis=1)
df_notices['doc'].head()

0    Lista liderada por Franck Ribéry conta com jog...
1    Com todos os titulares em campo, Atlético faz ...
2    Time mais disciplinado do Brasileiro, Atlético...
3    Elias explica como o Atlético deve jogar para ...
4    Por temor de punição, clubes da Série A lançam...
Name: doc, dtype: object

In [192]:
numbers = '0123456789'
stopwords = nltk.corpus.stopwords.words('portuguese')

def preprocessa(txt):
    txt = txt.lower()
    txt = ''.join([c for c in txt if c not in punctuation + numbers])
    txt = ' '.join([t for t in txt.split(' ') if t not in stopwords])
    txt = normalize('NFKD', txt).encode('ASCII', 'ignore').decode('utf-8')
    
    
    return txt

In [193]:
df_notices['doc'] = df_notices['doc'].apply(preprocessa)